In [1]:
import time
import enum
import math
import numpy as np
from Arm_Lib import Arm_Device
import cv2
import ipywidgets.widgets as widgets
import os
import glob
import dofbot_helper as dofbot

In [2]:
# Captuure and Display Image
def captureImage():
    for i in range(5):
        time.sleep(0.25)
        camera = cv2.VideoCapture(0)
        if camera is not None:
            ret, image = camera.read()
            if ret:
                return image

def captureImage_new():
    for i in range(5):
        time.sleep(0.25)
        camera = cv2.VideoCapture(0)
        if camera is not None:
            ret, image = camera.read()
            if ret:
                return ret, image
                
def displayImage(img, i):
    iw = widgets.Image(format='jpg', width=len(img), height=len(img[0]))  #This is for displaying the image in Jupyter
    #display(iw)
    iw.value = bytes(cv2.imencode('.jpg',img)[1])
    cv2.imwrite('SavedImage'+str(i)+'.jpg', img) # Need to iterate name
    


In [3]:
def move_joints(q, speedtime=100):
    for jnum in range(1, q.shape[0]+1):
        dofbot.moveJoint(jnum, q[jnum-1], speedtime)
        time.sleep(0.05)

In [4]:
def take_pictures():
    q_initial = np.array([90, 90, 45, 0, 90, 90])
    move_joints(q_initial, 500)

    for i in range(10):
        checkerboard_image = displayImage(captureImage(), i)
        # Move between iteration
        q_new = q_initial + np.array([0, 0, i*0.5, i*0.5, 0, 0])
        move_joints(q_new, 500)
        time.sleep(0.25)

In [5]:
q_initial = np.array([90, 90, 45, 0, 90, 90])
move_joints(q_initial, 500)

#take_pictures()

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((9*7,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:7].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('*.jpg')
img = cv2.imread('SavedImage0.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


for fname in images:
    img = cv2.imread(fname)
    # Find the chess board corners
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (9,7), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners, (20,20), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        #cv2.drawChessboardCorners(img, (7,9), corners2, ret)
        #cv2.imshow('img', img)
        #cv2.waitKey(500)
    
#cv2.destroyAllWindows()

# Calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print("DISTORTION COEFFICENTS")
print(dist)
# Undistortion
img = cv2.imread('SavedImage2.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
#print(newcameramtx)
#cv2.imwrite('test.jpg', img)
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png', dst)

# Re-projection error
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

q_initial = np.array([90, 90, 45, 0, 90, 90])
move_joints(q_initial, 500)
print("OLD CAMERA MATRIX")
print(mtx)
print("CAMERA MATRIX")
print(newcameramtx)
print("DISTORTION")
print(roi)

DISTORTION COEFFICENTS
[[-9.85210574e-01  1.81252783e+01  1.32663929e-02 -9.23384823e-03
  -2.37226129e+02]]
total error: 0.06228810574739997
OLD CAMERA MATRIX
[[1.16694501e+03 0.00000000e+00 3.38853767e+02]
 [0.00000000e+00 1.72086858e+03 2.22837231e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
CAMERA MATRIX
[[6.55945740e+02 0.00000000e+00 1.90471433e+02]
 [0.00000000e+00 1.09491992e+03 1.54717904e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
DISTORTION
(0, 13, 360, 305)


In [52]:
def dist():
    ret, frame = captureImage_new()
    #crop_img = frame[60:120, 0:160] # SEE IF THIS IS NECESSARY, MIGHT NOT BE 
    #print("Width:", len(frame))       # Width of image is 680
    #print("Height:", len(frame[0]))   # Height of image is 480

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0) # Constants might need to be changed here 
    ret, thresh = cv2.threshold(blur, 60, 255, cv2.THRESH_BINARY_INV)

    contours, hierarchy = cv2.findContours(thresh.copy(), 1, cv2.CHAIN_APPROX_NONE)

    # Select the largest contour and find its center
    if len(contours) > 0: 
        c = max(contours, key=cv2.contourArea)
        M = cv2.moments(c)
        # Get the center of each blob (not sure if this is the approach we want to take)
        
        #cx = int(M['m10']/M['m00'])
        #cy = int(M['m01']/M['m00'])

        #cv2.line(frame,(cx,0),(cx,720),(255,0,0),1)
        #cv2.line(frame,(0,cy),(1280,cy),(255,0,0),1)

        cv2.line(frame,(320,468),(320,478),(255,0,0),1)
        cv2.line(frame,(300,478),(340,478),(255,0,0),1)
        cv2.drawContours(frame, max(contours, key=cv2.contourArea), -1, (0, 255, 0), 1)

        '''
        if cx >= 120:
            print("Turn Left")
        if cx < 120 and cx > 50:
            print("On Track")
        if cx <= 50:
            print("Turn right")
        '''
    else:
        return 0, 0
        #print("I dont see the line")


    iw = widgets.Image(format='jpg', width=len(frame), height=len(frame[0]))
    iw.value = bytes(cv2.imencode('.jpg',frame)[1])
    cv2.imwrite('TEST'+'.jpg', frame)    

    # Find closest point along contour 
    end_effector = np.array([320,480])
    obstacle = max(contours, key=cv2.contourArea)
    obstacle = np.reshape(obstacle, (obstacle.shape[0], obstacle.shape[2]))

    min_dist = np.linalg.norm(obstacle[0,:]-end_effector)
    min_val = [] 
    
    if obstacle.shape[0] < 100:
        return 0, 0
    
    for val in obstacle: 
        dist = np.linalg.norm(val-end_effector)
        if dist < min_dist:
            min_dist = dist
            min_val = val
    
    #print(obstacle.shape)
    #print("The min distance is:", min_dist)
    #print("The min distance location is:", min_val)
    
    return min_dist, min_val

In [49]:
    
def attract_goal(q, qd, k):
    return -k*(q-qd)

def repel_goal(q, eta, rho0):
    d, val = 0, 0
    d, val = dist()
    if d == 0 or d > 200:
        return np.zeros((6,))
    else:
        # -eta*(1/d-1/rho0)*(-1/d^2)*2*(P-c)/norm(P-c);
        dif = np.array([3,0,0,0,0,0])
        return dif

def object_avoidance(q_prev, qf, step_size):
    lam = 0
    q_cur = q_prev
    flag = 0
    k = 2; # attractive gain
    eta = 0.001; # repulsive gain
    rho0 = 0.1; # buffer around obstacle where repulsive field is active
    
    for i in range(0, int(1/step_size)):
        q_cur = q_prev + step_size*attract_goal(q_prev, qf, k) + repel_goal(q_prev, eta, rho0)
        move_joints(q_cur)
        q_prev = q_cur
        lam += step_size
        
    return 
        

In [62]:
q_initial = np.array([90, 90, 15, 0, 90, 170])
q_final = np.array([100, 40, 60, 40, 90, 170])
move_joints(q_initial, 500)
#move_joints(q_final,500)

#q_new = q_initial - 0.03*(q_initial-q_final)
#print(q_new)
object_avoidance(q_initial, q_final, 0.03)

In [42]:
q_temp = np.array([93, 90, 15, 0, 90, 170])
move_joints(q_temp, 500)


In [36]:
min_dist, min_val = dist()

Turn Left
0
